# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

'''
Import note: The classroom currently uses sklearn version 0.19.
If you need to use an imputer, it is available in sklearn.preprocessing.Imputer,
instead of sklearn.impute as in newer versions of sklearn.
'''

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
# Load in the general demographics data.
azdias = pd.read_csv('Udacity_AZDIAS_Subset.csv', delimiter =';')

# Load in the feature summary file.
feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv', delimiter =';')

In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).
print('Feature Info')
print(feat_info.info())
print(feat_info)

In [ ]:
print()
print('Shape Azdias')
az_shape = azdias.shape
print(az_shape)
print('Describe Azdias')
print(azdias.describe())

In [ ]:
print()
print('First 10 rows:')
azdias.head(10)

> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
# Identify missing or unknown data values and convert them to NaNs.
# Feat_info: loop over all attributes
# identify for each attribute string for missing or unknown data
# replace these string values in azdias with NaN

num_attr = len(feat_info)

for i in range(num_attr):
    attr = feat_info['attribute'][i]
    values = feat_info['missing_or_unknown'][i]
    list_values = values[1:-1].split(',')
    print(len(list_values), list_values)

    for value in list_values:
        try:
            value = int(value)
            azdias[attr] = azdias[attr].replace([value],np.NaN)
        except ValueError:
            azdias[attr] = azdias[attr].replace([value],np.NaN)
    # Check for first 10 entries if replacement is correct
    print(azdias[attr].head(10))
    print()

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.
# Dictionary to store counts of missing data (percentage of values)
dict_nan = {}

for i in range(num_attr):
    attr = feat_info['attribute'][i]
    count = azdias[attr].isnull().sum()
    
    dict_nan[attr] = count*100 / az_shape[0]
    sorted_dict = sorted(dict_nan.values(), reverse = True)
    
print(dict_nan)
print()
print('Sorted list of percentage values:')
print(sorted_dict)

The sorted list of the percentage values shows that there are 6 columns with values which are much higher than 20%. Therefore, I will drop these six columns from the data set. From my point of view, this list shows better as a histogram which columns can be dropped.

In [ ]:
# Investigate patterns in the amount of missing data in each column.

list_drop_columns = []
for key in dict_nan:
    if dict_nan[key] > 20:
        list_drop_columns.append(key)

print('These columns should be dropped: ',list_drop_columns)



In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)
azdias.drop(columns = list_drop_columns, inplace = True)
print(azdias.head())


#### Discussion 1.1.2: Assess Missing Data in Each Column
The sorted list of the percentage values shows that there are 6 columns with values which are much higher than 20%. Therefore, I dropped these six columns from the data set. From my point of view, this list shows better as a histogram which columns can be dropped.

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# How much data is missing in each row of the dataset?
row_nans = azdias.isnull().sum(axis = 1)

row_nans.hist()
plt.ylabel('Counts')
plt.xlabel('Number of NaN')
plt.show()



In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.
# Adding the number of missing values as an additional column to the dataset
azdias['Missing values'] = row_nans

split_value = 30

az_mask_low_nan = azdias['Missing values'] < split_value
az_mask_high_nan = azdias['Missing values'] >= split_value

azdias_low_nan = azdias[az_mask_low_nan]
print(azdias_low_nan.shape)
azdias_high_nan = azdias[az_mask_high_nan]
print(azdias_high_nan.shape)

# The following analysis will be done with the data set with low Nan values
# remove column with number of missing values per row
azdias_low_nan.drop('Missing values', axis=1, inplace = True)
azdias_corr =  azdias_low_nan

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.
print('Analysis of dataset with low missing values:')
dict_nan_low = {}

column_names = azdias_low_nan.columns.tolist()

for i in range(len(column_names)):
    attr = column_names[i]
    count = azdias_low_nan[attr].isnull().sum()
    
    dict_nan_low[attr] = count
    sorted_dict_low = sorted(dict_nan_low.values(), reverse = True)
    
print(dict_nan_low)
print()
print('Sorted list of values:')
print(sorted_dict_low)

In [ ]:
# For some columns the same or very similar nummbers occur. 
# I check if these columns belong together:
step1 = 59000
step2 = 40000
step3 = 30000
step4 = 20000
for key in dict_nan_low:
    if dict_nan_low[key] > step1:
        print('Columns withmore than ', step1, 'missing values:', key)
for key in dict_nan_low:
    if step1 > dict_nan_low[key] > step2:
        print('Columns withmore than ', step2, 'missing values:', key)
for key in dict_nan_low:
    if step2 > dict_nan_low[key] > step3:
        print('Columns withmore than ', step3, 'missing values:', key)
for key in dict_nan_low:
    if step3 > dict_nan_low[key] > step4:
        print('Columns withmore than ', step4, 'missing values:', key)

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.
print('Analysis of dataset with high missing values:')
dict_nan_high = {}

column_names = azdias_high_nan.columns.tolist()

for i in range(len(column_names)):
    attr = column_names[i]
    count = azdias_high_nan[attr].isnull().sum()
    
    dict_nan_high[attr] = count
    sorted_dict_high = sorted(dict_nan_high.values(), reverse = True)
    
print(dict_nan_high)
print()
print('Sorted list of values:')
print(sorted_dict_high)

In [ ]:
# For some columns the same or very similar nummbers occur. 
# I check if these columns belong together:
step1 = 90000
step2 = 70000
step3 = 40000

for key in dict_nan_high:
    if dict_nan_high[key] > step1:
        print('Columns withmore than ', step1, 'missing values:', key)
for key in dict_nan_high:
    if step1 > dict_nan_high[key] > step2:
        print('Columns withmore than ', step2, 'missing values:', key)
for key in dict_nan_high:
    if step2 > dict_nan_high[key] > step3:
        print('Columns withmore than ', step3, 'missing values:', key)


#### Discussion 1.1.3: Assess Missing Data in Each Row

The assumption that some of the columns belong together and have the same or similar numbers of missing values is confirmed. For example we see this for KBA05_xxx and PLZ8_XXX in both datasets.

All columns with high values from the set with the low nan-values also occur in the set with high nan-values.

But the colunms with the highest value from the set with the high nan-values do not occur in the top rank of the other data set.

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?
print('Number of data types in original data set:')
print(feat_info['type'].value_counts())

print(feat_info)

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?
list_cat=[]
list_mix=[]

shape_feat = feat_info.shape
for i in range(shape_feat[0]):
    
    if feat_info['type'][i] == 'categorical' and feat_info['attribute'][i] not in list_drop_columns:
        list_cat.append(feat_info['attribute'][i])
     
    elif feat_info['type'][i] =='mixed'and feat_info['attribute'][i] not in list_drop_columns:
        list_mix.append(feat_info['attribute'][i])

print('List of categorical features:')
print(list_cat)
print()
print('List of mixed features:')
print(list_mix)


In [ ]:
# New list for categorical featues which should be encoded
list_cat_enco = []

for i in list_cat:
    if len(azdias_corr[i].value_counts()) >2:
        list_cat_enco.append(i)

print('List of categorical features for encoding:')
print(list_cat_enco)

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.

# Encoding feature by feature
for cat in list_cat_enco:
    # create dummies for new columns
    one_hot = pd.get_dummies(azdias_corr[cat], prefix = cat)
    # drop original column
    azdias_corr.drop(cat, axis=1, inplace = True)
    # add new columns
    azdias_corr = azdias_corr.join(one_hot)

print(azdias_corr.head(10))

#### Discussion 1.2.1: Re-Encode Categorical Features

First I analysed which categorical features have more than 2 values in the dataset and which are not already dropped. For these categories I did one hot encoding. Therefore, all different values are now represented with own columns in the dataset. 

Furthermore, I removed the original columns because their information is now available in the new columns.

Categorical features with only two different values are unchanged because they can be handled by the unsupervised learning algorithm.

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

### 1.18. PRAEGENDE_JUGENDJAHRE (copied from Data_Dictionary)
Dominating movement of person's youth (avantgarde vs. mainstream; east vs. west)
- -1: unknown
-  0: unknown
-  1: 40s - war years (Mainstream, E+W)
-  2: 40s - reconstruction years (Avantgarde, E+W)
-  3: 50s - economic miracle (Mainstream, E+W)
-  4: 50s - milk bar / Individualisation (Avantgarde, E+W)
-  5: 60s - economic miracle (Mainstream, E+W)
-  6: 60s - generation 68 / student protestors (Avantgarde, W)
-  7: 60s - opponents to the building of the Wall (Avantgarde, E)
-  8: 70s - family orientation (Mainstream, E+W)
-  9: 70s - peace movement (Avantgarde, E+W)
- 10: 80s - Generation Golf (Mainstream, W)
- 11: 80s - ecological awareness (Avantgarde, W)
- 12: 80s - FDJ / communist party youth organisation (Mainstream, E)
- 13: 80s - Swords into ploughshares (Avantgarde, E)
- 14: 90s - digital media kids (Mainstream, E+W)
- 15: 90s - ecological awareness (Avantgarde, E+W)

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
az_corr_shape = azdias_corr.shape
# derive intervall Variable for decade
# list for new column
col_empty = ['NaN'] * az_corr_shape[0]

# Adding new columns
azdias_corr['Decade'] = col_empty
azdias_corr['Stream'] = col_empty

# Dictionary Decade
dict_decade={1:'40s', 2:'40s', 3:'50s', 4: '50s', 5: '60s', 6:'60s', 7:'60s', 8:'70s', 9: '70s', 10:'80s',
             11:'80s', 12:'80s', 13: '13',  14: '90s', 15:'90s'}


dict_stream={1: 'mainstream', 3: 'mainstream', 5: 'mainstream', 8: 'mainstream', 10: 'mainstream', 12: 'mainstream',
                14: 'mainstream', 2: 'avantgarde', 4: 'avantgarde', 6: 'avantgarde', 7: 'avantgarde', 9: 'avantgarde', 
                 11: 'avantgarde', 13: 'avantgarde', 15: 'avantgarde'}

for i in dict_decade:
    # Masking for each value
    mask_decade = azdias_corr['PRAEGENDE_JUGENDJAHRE'] == i 
    # Assignment of values for decade
    azdias_corr['Decade'][mask_decade] = dict_decade[i]
    azdias_corr['Stream'][mask_decade] = dict_stream[i]

    
print(azdias_corr['PRAEGENDE_JUGENDJAHRE'].head())
print(azdias_corr['Decade'].head())
print(azdias_corr['Stream'].head())

In [ ]:
# One hot Encoding of Decade
# create dummies for new column
one_hot = pd.get_dummies(azdias_corr['Decade'], prefix = 'Youth')
# drop original column
azdias_corr.drop(['Decade'], axis=1, inplace = True)
# add new columns
azdias_corr = azdias_corr.join(one_hot)

# One hot Encoding of mainstream vs avantgarde
# create dummies for new column
one_hot = pd.get_dummies(azdias_corr['Stream'])
# drop original column
azdias_corr.drop(['Stream'], axis=1, inplace = True)
# add new columns
azdias_corr = azdias_corr.join(one_hot)

# Deleting of 'PRAEGENDE_JUGENDJAHRE'
azdias_corr.drop(['PRAEGENDE_JUGENDJAHRE'], axis=1, inplace = True)

### 4.3. CAMEO_INTL_2015
German CAMEO: Wealth / Life Stage Typology, mapped to international code
- -1: unknown
- 11: Wealthy Households - Pre-Family Couples & Singles
- 12: Wealthy Households - Young Couples With Children
- 13: Wealthy Households - Families With School Age Children
- 14: Wealthy Households - Older Families &  Mature Couples
- 15: Wealthy Households - Elders In Retirement
- 21: Prosperous Households - Pre-Family Couples & Singles
- 22: Prosperous Households - Young Couples With Children
- 23: Prosperous Households - Families With School Age Children
- 24: Prosperous Households - Older Families & Mature Couples
- 25: Prosperous Households - Elders In Retirement
- 31: Comfortable Households - Pre-Family Couples & Singles
- 32: Comfortable Households - Young Couples With Children
- 33: Comfortable Households - Families With School Age Children
- 34: Comfortable Households - Older Families & Mature Couples
- 35: Comfortable Households - Elders In Retirement
- 41: Less Affluent Households - Pre-Family Couples & Singles
- 42: Less Affluent Households - Young Couples With Children
- 43: Less Affluent Households - Families With School Age Children
- 44: Less Affluent Households - Older Families & Mature Couples
- 45: Less Affluent Households - Elders In Retirement
- 51: Poorer Households - Pre-Family Couples & Singles
- 52: Poorer Households - Young Couples With Children
- 53: Poorer Households - Families With School Age Children
- 54: Poorer Households - Older Families & Mature Couples
- 55: Poorer Households - Elders In Retirement
- XX: unknown

In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.
print(azdias_corr['CAMEO_INTL_2015'].head())

# Adding new columns
azdias_corr['Wealth'] = col_empty
azdias_corr['Life_stage'] = col_empty

# Dictionary Wealth
dict_wealth = {11: 'wealthy', 12: 'wealthy', 13: 'wealthy', 14: 'wealthy', 15: 'wealthy',
              21: 'prosperous', 22: 'prosperous', 23: 'prosperous', 24: 'prosperous', 25: 'prosperous',
              31: 'comfortable', 32: 'comfortable', 33: 'comfortable', 34: 'comfortable', 35: 'comfortable',
              41: 'less affluent', 42: 'less affluent', 43: 'less affluent', 44: 'less affluent', 45: 'less affluent',
              51: 'poorer', 52: 'poorer', 53: 'poorer', 54: 'poorer', 55: 'poorer' }

# Dictionary Life stage
dict_lstage = {11: 'Pre-Family Couples & Singles', 21: 'Pre-Family Couples & Singles', 31: 'Pre-Family Couples & Singles',
              41: 'Pre-Family Couples & Singles', 51: 'Pre-Family Couples & Singles',
              12: 'Young Couples With Children', 22: 'Young Couples With Children', 32: 'Young Couples With Children',
              42: 'Young Couples With Children', 52: 'Young Couples With Children',
              13: 'Families With School Age Children', 23: 'Families With School Age Children', 
              33: 'Families With School Age Children', 43: 'Families With School Age Children',
              53: 'Families With School Age Children',
              14: 'Older Families & Mature Couples', 24: 'Older Families & Mature Couples', 34: 'Older Families & Mature Couples',
              44: 'Older Families & Mature Couples', 54: 'Older Families & Mature Couples',
              15: 'Elders In Retirement', 25: 'Elders In Retirement', 35: 'Elders In Retirement', 45: 'Elders In Retirement',
              55: 'Elders In Retirement'}

for i in dict_wealth:
    # Masking for each value
    mask_cameo = azdias_corr['CAMEO_INTL_2015'] == str(i)
    
    # Assignment of values for wealth and life stage
    azdias_corr['Wealth'][mask_cameo] = dict_wealth[i]
    azdias_corr['Life_stage'][mask_cameo] = dict_lstage[i]


print(azdias_corr['Wealth'].head())
print(azdias_corr['Life_stage'].head())

In [ ]:
# Encoding Wealth and family stage
list_cat =['Wealth','Life_stage']
for cat in list_cat:
    # create dummies for new columns
    one_hot = pd.get_dummies(azdias_corr[cat], prefix = cat)
    # drop original column
    azdias_corr.drop(cat, axis=1, inplace = True)
    # add new columns
    azdias_corr = azdias_corr.join(one_hot)

# Deleting of 'CAMEO_INTL_2015' from the dataset
azdias_corr.drop('CAMEO_INTL_2015', axis=1, inplace = True)

In [ ]:
# Investigation of other mixed categories
print('List of mixed features:')
print(list_mix)
print()


### 1.11. LP_LEBENSPHASE_FEIN
Life stage, fine scale
-  1: single low-income earners of younger age
-  2: single low-income earners of middle age
-  3: single average earners of younger age
-  4: single average earners of middle age
-  5: single low-income earners of advanced age
-  6: single low-income earners at retirement age
-  7: single average earners of advanced age
-  8: single average earners at retirement age
-  9: single independent persons
- 10: wealthy single homeowners
- 11: single homeowners of advanced age
- 12: single homeowners at retirement age
- 13: single top earners of higher age
- 14: low-income and average earner couples of younger age
- 15: low-income earner couples of higher age
- 16: average earner couples of higher age
- 17: independent couples
- 18: wealthy homeowner couples of younger age
- 19: homeowner couples of higher age
- 20: top earner couples of higher age
- 21: single parent low-income earners
- 22: single parent average earners
- 23: single parent high-income earners
- 24: low-income earner families
- 25: average earner families
- 26: independent families
- 27: homeowner families
- 28: top earner families
- 29: low-income earners of younger age from multiperson households
- 30: average earners of younger age from multiperson households
- 31: low-income earners of higher age from multiperson households
- 32: average earners of higher age from multiperson households
- 33: independent persons of younger age from multiperson households
- 34: homeowners of younger age from multiperson households
- 35: top earners of younger age from multiperson households
- 36: independent persons of higher age from multiperson households
- 37: homeowners of advanced age from multiperson households
- 38: homeowners at retirement age from multiperson households
- 39: top earners of middle age from multiperson households
- 40: top earners at retirement age from multiperson households

### 1.12. LP_LEBENSPHASE_GROB
Life stage, rough scale
-  1: single low-income and average earners of younger age
-  2: single low-income and average earners of higher age
-  3: single high-income earners
-  4: single low-income and average-earner couples
-  5: single high-income earner couples
-  6: single parents
-  7: single low-income and average earner families
-  8: high-income earner families
-  9: average earners of younger age from multiperson households
- 10: low-income and average earners of higher age from multiperson households
- 11: high-income earners of younger age from multiperson households
- 12: high-income earners of higher age from multiperson households

### 3.7. WOHNLAGE
Neighborhood quality (or rural flag)
- -1: unknown
-  0: no score calculated
-  1: very good neighborhood
-  2: good neighborhood
-  3: average neighborhood
-  4: poor neighborhood
-  5: very poor neighborhood
-  7: rural neighborhood
-  8: new building in rural neighborhood

### 8.6. PLZ8_BAUMAX
Most common building type within the PLZ8 region
- -1: unknown
-  0: unknown
-  1: mainly 1-2 family homes
-  2: mainly 3-5 family homes
-  3: mainly 6-10 family homes
-  4: mainly 10+ family homes
-  5: mainly business buildings

In [ ]:
print(azdias_corr['WOHNLAGE'].head())
print(azdias_corr['PLZ8_BAUMAX'].head())

In [ ]:
# Handling of the remaining mixed categories
# Content of features "LP Lebensphase fein/grob" is already covered by content of CAMEO_INTL_2015
# and can be removed.
azdias_corr.drop('LP_LEBENSPHASE_FEIN', axis=1, inplace = True)
azdias_corr.drop('LP_LEBENSPHASE_GROB', axis=1, inplace = True)

# Features 'WOHNLAGE' and 'PLZ8_BAUMAX' can be handled as ordinal numbers
# They are already float and need no change.

#### Discussion 1.2.2: Engineer Mixed-Type Features

### Praegende Jugendjahre (english: forming youth age)
This feature is a combination of the youth generation which the person belonged to and a categorization of these youth generation into avantgarde or mainstream.

Even if the decade of birth could be considered as a number, the original feature delivers information about social aspects. Therefore, I handled it as a categorical feature and made an one-hot encoding for the youth generation. The information mainstream vs. avantgarde was also one hot encoded.

### CAMEO_INTL_2015
This combination is a combination of wealth and life stage. I splitted the feature into these two new features. For both I made an one hot encoding.

### Conclusion
All new features include interesting information and therefore, I kept them in the data set.


#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# If there are other re-engineering tasks you need to perform, make sure you
# take care of them here. (Dealing with missing data will come in step 2.1.)
print('List of categorical features for encoding:')
print(list_cat_enco)
print()
print('List of mixed features:')
list_mix.remove('WOHNLAGE')
list_mix.remove('PLZ8_BAUMAX' )
print(list_mix)
print()
# Derived categories from mixed features
list_derived = ['Decade', 'Stream', 'Wealth', 'Life_stage', 'Missing values' ]
print('Derived categories from mixed features:') 
print(list_derived)
print()




In [ ]:
# Combination of all these features
list_not_used_feat = list_cat_enco + list_mix + list_derived
print(list_not_used_feat)

# List of all column names:
list_columns = azdias_corr.columns
print(list_columns)

print()
print('Check of features columns:')
for feat in list_not_used_feat:
    if feat in list_columns:
        print('The feature', feat, 'is still in the data set!')
    else:
        print('ok')

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.
list_dtype = []
for feat in list_columns:
        list_dtype.append(azdias_corr[feat].dtype)
        if azdias_corr[feat].dtype == 'object':
            print(feat, ' is still object')

counts = pd.Series(list_dtype)
print(counts.value_counts())       

In [ ]:
# Change of OST_WEST_KZ to integer
dict_ost_west = {'W': 1, 'O': 2}

for i in dict_ost_west:
    # Masking each value
    mask_ow = azdias_corr['OST_WEST_KZ']
    azdias_corr['OST_WEST_KZ'] = int(dict_ost_west[i])    

list_dtype = []
for feat in list_columns:
        list_dtype.append(azdias_corr[feat].dtype)
        if azdias_corr[feat].dtype == 'object':
            print(feat, ' is still object')

counts = pd.Series(list_dtype)
print(counts.value_counts())

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_data(df):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    
    # Put in code here to execute all main cleaning steps:
    # convert missing value codes into NaNs, ...
    for value in list_values:
        try:
            value = int(value)
            df[attr] = df[attr].replace([value],np.NaN)
        except ValueError:
            df[attr] = df[attr].replace([value],np.NaN)
    
    # remove selected columns 
    list_drop_columns = ['AGER_TYP', 'GEBURTSJAHR', 'TITEL_KZ', 'ALTER_HH', 'KK_KUNDENTYP', 'KBA05_BAUMAX']
    df.drop(columns = list_drop_columns, inplace = True)
    
    # remove selected rows 
    df['Missing values'] = row_nans
    split_value = 30
    df_mask_low_nan = df['Missing values'] < split_value
    #az_mask_high_nan = azdias['Missing values'] >= split_value

    df_low_nan = df[df_mask_low_nan]
    df_low_nan.drop('Missing values', axis=1, inplace = True)
    df_corr =  df_low_nan
    
    # select, re-encode, and engineer column values.
    list_cat_enco = ['CJT_GESAMTTYP', 'FINANZTYP', 'GFK_URLAUBERTYP', 'LP_FAMILIE_FEIN', 'LP_FAMILIE_GROB', 'LP_STATUS_FEIN', 'LP_STATUS_GROB', 
    'NATIONALITAET_KZ', 'SHOPPER_TYP', 'ZABEOTYP', 'GEBAEUDETYP', 'CAMEO_DEUG_2015', 'CAMEO_DEU_2015']
    
 
    # Encoding feature by feature
    for cat in list_cat_enco:
        # create dummies for new columns
        one_hot = pd.get_dummies(df_corr[cat], prefix = cat)
        # drop original column
        df.drop(cat, axis=1, inplace = True)
        # add new columns
        df_corr = df_corr.join(one_hot)
    
    df_corr_shape = df_corr.shape
    # derive intervall Variable for decade
    # list for new column
    col_empty = ['NaN'] * df_corr_shape[0]
    # Adding new columns
    df_corr['Decade'] = col_empty
    df_corr['Stream'] = col_empty

    # Dictionary Decade
    dict_decade={1:'40s', 2:'40s', 3:'50s', 4: '50s', 5: '60s', 6:'60s', 7:'60s', 8:'70s', 9: '70s', 10:'80s',
                 11:'80s', 12:'80s', 13: '13',  14: '90s', 15:'90s'}


    dict_stream={1: 'mainstream', 3: 'mainstream', 5: 'mainstream', 8: 'mainstream', 10: 'mainstream', 12: 'mainstream',
                    14: 'mainstream', 2: 'avantgarde', 4: 'avantgarde', 6: 'avantgarde', 7: 'avantgarde', 9: 'avantgarde', 
                     11: 'avantgarde', 13: 'avantgarde', 15: 'avantgarde'}

    for i in dict_decade:
        # Masking for each value
        mask_decade = df_corr['PRAEGENDE_JUGENDJAHRE'] == i 
        # Assignment of values for decade
        df_corr['Decade'][mask_decade] = dict_decade[i]
        df_corr['Stream'][mask_decade] = dict_stream[i]     
        
    # One hot Encoding of Decade
    # create dummies for new column
    one_hot = pd.get_dummies(df_corr['Decade'], prefix = 'Youth')
    # drop original column
    df_corr.drop(['Decade'], axis=1, inplace = True)
    # add new columns
    df_corr = df_corr.join(one_hot)

    # One hot Encoding of mainstream vs avantgarde
    # create dummies for new column
    one_hot = pd.get_dummies(df_corr['Stream'])
    # drop original column
    df_corr.drop(['Stream'], axis=1, inplace = True)
    # add new columns
    df_corr = df_corr.join(one_hot)

    # Deleting of 'PRAEGENDE_JUGENDJAHRE'
    df_corr.drop(['PRAEGENDE_JUGENDJAHRE'], axis=1, inplace = True )       

    # Adding new columns
    df_corr['Wealth'] = col_empty
    df_corr['Life_stage'] = col_empty

    # Dictionary Wealth
    dict_wealth = {11: 'wealthy', 12: 'wealthy', 13: 'wealthy', 14: 'wealthy', 15: 'wealthy',
                  21: 'prosperous', 22: 'prosperous', 23: 'prosperous', 24: 'prosperous', 25: 'prosperous',
                  31: 'comfortable', 32: 'comfortable', 33: 'comfortable', 34: 'comfortable', 35: 'comfortable',
                  41: 'less affluent', 42: 'less affluent', 43: 'less affluent', 44: 'less affluent', 45: 'less affluent',
                  51: 'poorer', 52: 'poorer', 53: 'poorer', 54: 'poorer', 55: 'poorer' }

    # Dictionary Life stage
    dict_lstage = {11: 'Pre-Family Couples & Singles', 21: 'Pre-Family Couples & Singles', 31: 'Pre-Family Couples & Singles',
                  41: 'Pre-Family Couples & Singles', 51: 'Pre-Family Couples & Singles',
                  12: 'Young Couples With Children', 22: 'Young Couples With Children', 32: 'Young Couples With Children',
                  42: 'Young Couples With Children', 52: 'Young Couples With Children',
                  13: 'Families With School Age Children', 23: 'Families With School Age Children', 
                  33: 'Families With School Age Children', 43: 'Families With School Age Children',
                  53: 'Families With School Age Children',
                  14: 'Older Families & Mature Couples', 24: 'Older Families & Mature Couples', 34: 'Older Families & Mature Couples',
                  44: 'Older Families & Mature Couples', 54: 'Older Families & Mature Couples',
                  15: 'Elders In Retirement', 25: 'Elders In Retirement', 35: 'Elders In Retirement', 45: 'Elders In Retirement',
                  55: 'Elders In Retirement'}
                 
    for i in dict_wealth:
        # Masking for each value
        mask_cameo = df_corr['CAMEO_INTL_2015'] == str(i)

        # Assignment of values for wealth and life stage
        df_corr['Wealth'][mask_cameo] = dict_wealth[i]
        df_corr['Life_stage'][mask_cameo] = dict_lstage[i]

    # Encoding Wealth and family stage
    list_cat =['Wealth','Life_stage']
    for cat in list_cat:
        # create dummies for new columns
        one_hot = pd.get_dummies(df_corr[cat], prefix = cat)
        # drop original column
        df_corr.drop(cat, axis=1, inplace = True)
        # add new columns
        df_corr = df_corr.join(one_hot)

    # Deleting of 'CAMEO_INTL_2015' from the dataset
    df_corr.drop('CAMEO_INTL_2015', axis=1, inplace = True)
                 
    # Content of features "LP Lebensphase fein/grob" is already covered by content of CAMEO_INTL_2015
    # and can be removed.
    df_corr.drop('LP_LEBENSPHASE_FEIN', axis=1, inplace = True)
    df_corr.drop('LP_LEBENSPHASE_GROB', axis=1, inplace = True)                 
                 

    # If there are other re-engineering tasks you need to perform, make sure you
    # take care of them here. (Dealing with missing data will come in step 2.1.)
    list_mix.remove('WOHNLAGE')
    # Derived categories from mixed features
    list_derived = ['Decade', 'Stream', 'Wealth', 'Life_stage' ]

    # Combination of all these features
    list_not_used_feat = list_cat_enco + list_mix + list_derived

    # List of all column names:
    list_columns = df_corr.columns
                 
    for feat in list_not_used_feat:
        if feat in list_columns:
            print('The feature', feat, 'is still in the data set!')

    # Change of OST_WEST_KZ to integer
    dict_ost_west = {'W': 1, 'O': 2}

    for i in dict_ost_west:
        # Masking each value
        mask_ow = df_corr['OST_WEST_KZ']
        df_corr['OST_WEST_KZ'] = int(dict_ost_west[i])    

    list_dtype = []
    for feat in list_columns:
            list_dtype.append(df_corr[feat].dtype)
            if df_corr[feat].dtype == 'object':
                print(feat, ' is still object')

    counts = pd.Series(list_dtype)
    print(counts.value_counts())    
    # Replacing remaining NaN values with 0
    df_corr.fillna(0, inplace = True)
    # Return the cleaned dataframe.
    return df_corr
    

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](https://scikit-learn.org/0.16/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
# If you've not yet cleaned the dataset of all NaN values, then investigate and
# do that now.
# Replacing remaining NaN values with 0
azdias_corr.fillna(0, inplace = True)
print('Number of NaN values: ',azdias_corr.isnull().any().sum())



In [ ]:
# Apply feature scaling to the general population demographics data.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean = True, with_std = True)
azdias_scaled = scaler.fit_transform(azdias_corr)

print(azdias_scaled.shape)

### Discussion 2.1: Apply Feature Scaling

I replaced already the columns and rows with many NaN values. Therefore, I just replaced the remaining NaN values by zero.

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.

from sklearn.decomposition import PCA
pca = PCA(n_components = 50)
pca.fit(azdias_scaled)


In [ ]:
# This helper function was provided for the lesson about PCA
# I modified it a little bit

def scree_plot(pca):
    '''
    Creates a scree plot associated with the principal components 
    
    INPUT: pca - the result of instantian of PCA in scikit learn
            
    OUTPUT:
            None
    '''
    num_components=len(pca.explained_variance_ratio_)
    ind = np.arange(num_components)
    vals = pca.explained_variance_ratio_
 
    plt.figure(figsize=(10, 6))
    ax = plt.subplot(111)
    cumvals = np.cumsum(vals)
    ax.bar(ind, vals)
    ax.plot(ind, cumvals)
    for i in range(num_components):
        ax.annotate(r"%s%%" % ((str(vals[i]*100)[:4])), (ind[i]+0.2, vals[i] + 0.01), va="bottom", ha="center", fontsize=12,
                   rotation = 90)
 
    ax.xaxis.set_tick_params(width=0)
    ax.yaxis.set_tick_params(width=2, length=12)
 
    ax.set_xlabel("Principal Component")
    ax.set_ylabel("Variance Explained (%)")
    plt.title('Explained Variance Per Principal Component')

In [ ]:
# Investigate the variance accounted for by each principal component.
scree_plot(pca)


In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain
# Because of technical problems I reduced already the number of components.
# pca = PCA(n_components = 20)
# pca.fit(azdias_scaled)

scree_plot(pca)

### Discussion 2.2: Perform Dimensionality Reduction

During the last steps technical problems occured. It was not possible to do the PCA with all components. The connection to the server was lost with more than 50 components. 

I decided to reduce the number of components because only the first 20 explain more than 1% of the total variance.

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
feature_list = azdias_corr.columns
num_features = len(feature_list)

def weight_feature_mapping(comp):
    dict_feature_weight = {}
    # Map each weight to the corresponding feature name
    for i in range(num_features):
        dict_feature_weight[feature_list[i]] = pca.components_[comp,i]
    
    # sort features according to weight, result is a list of tuples (feature, weight)
    sortedDict = sorted(dict_feature_weight.items(), key = lambda x: x[1])

    return sortedDict

In [ ]:
# Map weights for the first principal component to corresponding feature names
# and then print the linked values, sorted by weight.
# HINT: Try defining a function here or in a new cell that you can reuse in the
# other cells.

print('Largest negative weigths:')
print(weight_feature_mapping(0)[:5])
print()
print('Largest positive weigths:')
print(weight_feature_mapping(0)[-5:])

In [ ]:
# Map weights for the second principal component to corresponding feature names
# and then print the linked values, sorted by weight.

print('Largest negative weigths:')
print(weight_feature_mapping(1)[:5])
print()
print('Largest positive weigths:')
print(weight_feature_mapping(1)[-5:])

In [ ]:
# Map weights for the third principal component to corresponding feature names
# and then print the linked values, sorted by weight.

print('Largest negative weigths:')
print(weight_feature_mapping(2)[:5])
print()
print('Largest positive weigths:')
print(weight_feature_mapping(2)[-5:])

### Discussion 2.3: Interpret Principal Components

We see that the features with the largest weights (positive or negative) are related to
- financial behaviour (negative: "FINANZ_MINIMALIST, 'FINANZ_SPARER', 'FINANZ_UNAUFFAELLIGER', ..)
- residential area ('KBA05_...')
- income (HH_EINKOMMEN_SCORE
- social status ('LP_STATUS_GROB_1.0')

From my perspective these feature can describe well a typical consumer.


## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# Transform scaled data
pca_dataset = pca.transform(azdias_scaled)



In [ ]:
# create test_dataset, only used for development of code
test_samples = 50_000
knn_testset = pca_dataset[:test_samples, :]
print(knn_testset.shape)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# data = knn_testset  # used for code development
data = pca_dataset
n_points = data.shape[1]

centers = []
scores = []

# Over a number of different cluster counts...
for k in range(10, 30):

    # run k-means clustering on the data and...
    kmeans_k = KMeans(k)
    model = kmeans_k.fit(data)
    #labels = model.predict(data)
    # compute the average within-cluster distances.
    result = abs(model.score(data))/n_points
    centers.append(k)
    scores.append(result)
    
    

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.

plt.xlabel('Number of clusters');
plt.ylabel('Distance per data point');
plt.title('Distance vs. K');

plt.plot(centers, scores)
plt.show() 

In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.



### Discussion 3.1: Apply Clustering to General Population

(Double-click this cell and replace this text with your own text, reporting your findings and decisions regarding clustering. Into how many clusters have you decided to segment the population?)

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.

# customers = 

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.



### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.



In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?



In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?



### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.